In [13]:
import os
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from PIL import Image

import src.io as sio
import src.preprocessing as spp
import src.fitting as sft
import src.qudi_fit_wrapper as qft
import src.helpers as shp

from pprint import pprint

In [2]:
DATA_FOLDER = "20220112_SingleNV_Membrane_RT"
DATA_FOLDERPATH, FIGURE_FOLDERPATH = sio.get_data_and_figure_paths(DATA_FOLDER)

src.io :: INFO :: qudiamond folderpath is \\kernix\qudiamond\Data\20220112_SingleNV_Membrane_RT\
src.io :: INFO :: Figure folderpath is C:/Nextcloud\Data_Analysis\20220112_SingleNV_Membrane_RT


# Rabi

In [3]:
rabi = sio.read_pulsed_measurement_data(DATA_FOLDERPATH, measurement_str="rabi")

\\kernix\qudiamond\Data\20220112_SingleNV_Membrane_RT\Transmission\PowerVariation: : 69it [00:04, 14.88it/s]              
20220208-1950-35: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:12<00:00,  1.16s/it]


In [4]:
rabi.keys()

dict_keys(['20220208-1055-10', '20220208-1401-00', '20220208-1453-21', '20220208-1600-40', '20220208-1623-58', '20220208-1629-50', '20220208-1641-00', '20220208-1647-50', '20220208-1921-36', '20220208-1932-34', '20220208-1950-35'])

In [33]:
qft.get_fits()

odict_keys(['biexponential', 'decayexponential', 'decayexponentialstretched', 'gaussian', 'gaussiandouble', 'gaussianlinearoffset', 'hyperbolicsaturation', 'linear', 'lorentzian', 'lorentziandouble', 'lorentziantriple', 'poissonian', 'poissoniandouble', 'sine', 'sinedouble', 'sinedoublewithexpdecay', 'sinedoublewithtwoexpdecay', 'sineexponentialdecay', 'sinestretchedexponentialdecay', 'sinetriple', 'sinetriplewithexpdecay', 'sinetriplewiththreeexpdecay'])

In [5]:
rabi1 = rabi.copy()
del rabi1["20220208-1055-10"]
del rabi1["20220208-1401-00"]
del rabi1["20220208-1950-35"]

In [6]:
rabi1.keys()

dict_keys(['20220208-1453-21', '20220208-1600-40', '20220208-1623-58', '20220208-1629-50', '20220208-1641-00', '20220208-1647-50', '20220208-1921-36', '20220208-1932-34'])

In [77]:
frequencies, powers = [], []

In [78]:
%matplotlib widget

fig, ax = plt.subplots(nrows=len(rabi1), sharex=True, figsize=(4, 8))

for idx, key in enumerate(rabi1.keys()):
    df = rabi1[key]["pulsed_measurement"]["data"]
    filename = rabi1[key]["pulsed_measurement"]["filename"]
    power = re.search('rabi_(.*)dBm', filename).group(1)
    x = df["Controlled variable(s)"] * 1e9
    y = df["Signal"] / df["Signal"].max() 
    ax[idx].plot(x, y, ".", mfc="white")
    
    fit_x, fit_y, result = qft.perform_fit(x, y, "sineexponentialdecay")
    pi_length = 1 / result.params["frequency"].value
    pi_length_err = pi_length * (result.params["frequency"].stderr / result.params["frequency"].value)
        
    powers.append(shp.decibelm_to_watts(float(power)))
    frequencies.append(result.params["frequency"].value)
    
    ax[idx].plot(fit_x, fit_y)
    
    ax[idx].set_title(r"$P=$" + f"{power} dBm, " + r"$\tau_{\pi}=$" + f"{int(pi_length)} ± {int(pi_length_err)}ns ")

fig.tight_layout()
ax[-1].set_xlabel("Time (ns)")
#sio.save_figures("rabi_comparison", FIGURE_FOLDERPATH)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 59.7222222222222, 'Time (ns)')

In [81]:
%matplotlib widget

fig, ax = plt.subplots(figsize=(5, 4))
x = np.sqrt(powers)
y = np.array(frequencies) * 1e3
ax.plot(x, y, "o", mfc="white")
ax.set_xlabel(r"Power ($\sqrt{W}$)")
ax.set_ylabel("Frequency (MHz) ")

fit_x, fit_y, result = qft.perform_fit(x, y, "linear")
ax.plot(fit_x, fit_y)
fig.tight_layout()
# sio.save_figures("rabi_power_dep", FIGURE_FOLDERPATH)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

src.io :: INFO :: Figure folderpath is C:/Nextcloud\Data_Analysis\20220112_SingleNV_Membrane_RT
src.io :: INFO :: Saving 'rabi_power_dep' to 'C:/Nextcloud\Data_Analysis\20220112_SingleNV_Membrane_RT'


In [82]:
type(result)

lmfit.model.ModelResult

In [ ]:
print(result.params["frequency"].value)

In [54]:
%matplotlib widget

fig, ax = plt.subplots()

timetrace1 = rabi["20220208-1950-35"]["raw_timetrace"]["data"][1]
timetrace2 = rabi["20220208-1950-35"]["raw_timetrace"]["data"][0]

ax.plot(timetrace1, linewidth=0.2, label=r"State $|0\rangle$")
ax.plot(timetrace2, linewidth=0.2, label=r"State $|1\rangle$")
ax.plot(timetrace1 - timetrace2, linewidth=0.2, label=r"Signal ($|0\rangle - |1\rangle$)")

ax.axvspan(160, 180+200, alpha=0.3, color="C7", linewidth=0)
ax.set_xlabel("Time (ns)")
ax.set_ylabel("Photons")

ax.legend()
# sio.save_figures("state_timetrace", FIGURE_FOLDERPATH)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …